In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import transformers
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from torch import nn
import torch.nn.functional as F
#from tqdm import trange
from tqdm.notebook import tqdm, trange
import os
from transformers import AdamW, get_linear_schedule_with_warmup
import nlpaug
from tcn import TemporalConvNet

#DECLARE USER HERE
#Hardeep: set to 0
#Edmund: set to 1
set_user = 0

cuda


**SETTING ENVIRONMENT**

In [2]:
# PATH = None
# set_user = 3
# if set_user == 0:
#   PATH = '/content/gdrive/MyDrive/182proj/'
# elif set_user == 1:
#   PATH = '/content/gdrive/MyDrive/nlp_proj/'
# import sys
# import os
# sys.path.append(os.path.abspath(PATH))
PATH = 'C:/Users/Hardeep/Desktop/nlp_proj/'


### Data Loading/Augmentations

***ONLY RUN BELOW CELL ONCE OR ELSE DATA WILL BE CORRUPTED AND YOU WILL NEED TO RESTART THE RUNTIME***

In [3]:
data = pd.read_json(PATH + 'yelp_review_training_dataset.jsonl', lines=True)

delete_aug_data = pd.read_csv(PATH + 'augmentations/delete_aug.csv', header=None)
swap_aug_data = pd.read_csv(PATH + 'augmentations/swap_aug.csv', header=None)
typo_aug_data = pd.read_csv(PATH + 'augmentations/typo_aug.csv', header=None)
synoynm_aug_data = pd.read_csv(PATH + 'augmentations/synonym_aug.csv', header=None)

delete_aug_data.columns = delete_aug_data.iloc[0]
delete_aug_data = delete_aug_data[1:]
delete_aug_data['stars'] = delete_aug_data['stars'].astype('int64')

swap_aug_data.columns = swap_aug_data.iloc[0]
swap_aug_data = swap_aug_data[1:]
swap_aug_data['stars'] = swap_aug_data['stars'].astype('int64')

typo_aug_data.columns = typo_aug_data.iloc[0]
typo_aug_data = typo_aug_data[1:]
typo_aug_data['stars'] = typo_aug_data['stars'].astype('int64')

synoynm_aug_data.columns = synoynm_aug_data.iloc[0]
synoynm_aug_data = synoynm_aug_data[1:]
synoynm_aug_data['stars'] = synoynm_aug_data['stars'].astype('int64')

C:\Users\Hardeep\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
delete_aug_data.shape, swap_aug_data.shape, typo_aug_data.shape, synoynm_aug_data.shape, data.shape

((533581, 2), (533581, 2), (533581, 2), (533581, 2), (533581, 3))

TRAIN TEST SPLIT

In [5]:
train_test_size = 0.1
val_test_size = 0.4
random_state = 123

test_size = 1000

X = data['text'][:test_size]
y = data['stars'][:test_size]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=train_test_size, random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=val_test_size, random_state=random_state)

delete_X = delete_aug_data['text'][:test_size]
delete_y = delete_aug_data['stars'][:test_size]
delete_X_train, delete_X_test, delete_y_train, delete_y_test = train_test_split(delete_X, delete_y, test_size=train_test_size, random_state=random_state)
delete_X_val, delete_X_test, delete_y_val, delete_y_test = train_test_split(delete_X_test, delete_y_test, test_size=val_test_size, random_state=random_state)

swap_X = swap_aug_data['text'][:test_size]
swap_y = swap_aug_data['stars'][:test_size]
swap_X_train, swap_X_test, swap_y_train, swap_y_test = train_test_split(swap_X, swap_y, test_size=train_test_size, random_state=random_state)
swap_X_val, swap_X_test, swap_y_val, swap_y_test = train_test_split(swap_X_test, swap_y_test, test_size=val_test_size, random_state=random_state)

typo_X = typo_aug_data['text'][:test_size]
typo_y = typo_aug_data['stars'][:test_size]
typo_X_train, typo_X_test, typo_y_train, typo_y_test = train_test_split(typo_X, typo_y, test_size=train_test_size, random_state=random_state)
typo_X_val, typo_X_test, typo_y_val, typo_y_test = train_test_split(typo_X_test, typo_y_test, test_size=val_test_size, random_state=random_state)

synoynm_X = synoynm_aug_data['text'][:test_size]
synoynm_y = synoynm_aug_data['stars'][:test_size]
synoynm_X_train, synoynm_X_test, synoynm_y_train, synoynm_y_test = train_test_split(synoynm_X, synoynm_y, test_size=train_test_size, random_state=random_state)
synoynm_X_val, synoynm_X_test, synoynm_y_val, synoynm_y_test = train_test_split(synoynm_X_test, synoynm_y_test, test_size=val_test_size, random_state=random_state)

In [6]:
# Returns a tuple of 3 things:
# 0: dictionary of input_ids, token_type_ids, attention_masks
# 1: Review itself
# 2: Rating
class ProcessData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len, labels):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = labels
    
    def __getitem__(self, index):
        review = self.data[index]
        label = self.labels[index]
        return self.tokenizer.encode_plus(review, max_length=self.max_len, padding='max_length', return_attention_mask=True, return_tensors='pt', truncation=True), review, torch.tensor([label-1]).to(torch.long)
    
    def __len__(self):
        review_length = len(self.data)
        return review_length


tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
# TODO: DONT HARDCODE 150
max_length = 150
tokenized_training_data = ProcessData(X_train.to_numpy().tolist(), tokenizer, max_length, y_train.to_numpy())
tokenized_validation_data = ProcessData(X_val.to_numpy().tolist(), tokenizer, max_length, y_val.to_numpy())
tokenized_test_data = ProcessData(X_test.to_numpy().tolist(), tokenizer, max_length, y_test.to_numpy())

tokenized_training_data_typo = ProcessData(typo_X_train.to_numpy().tolist(), tokenizer, max_length, y_train.to_numpy())
tokenized_validation_data_typo = ProcessData(typo_X_val.to_numpy().tolist(), tokenizer, max_length, y_val.to_numpy())
tokenized_test_data_typo = ProcessData(typo_X_test.to_numpy().tolist(), tokenizer, max_length, y_test.to_numpy())

tokenized_training_data_swap = ProcessData(swap_X_train.to_numpy().tolist(), tokenizer, max_length, y_train.to_numpy())
tokenized_validation_data_swap = ProcessData(swap_X_val.to_numpy().tolist(), tokenizer, max_length, y_val.to_numpy())
tokenized_test_data_swap = ProcessData(swap_X_test.to_numpy().tolist(), tokenizer, max_length, y_test.to_numpy())

tokenized_training_data_delete = ProcessData(delete_X_train.to_numpy().tolist(), tokenizer, max_length, y_train.to_numpy())
tokenized_validation_data_delete = ProcessData(delete_X_val.to_numpy().tolist(), tokenizer, max_length, y_val.to_numpy())
tokenized_test_data_delete = ProcessData(delete_X_test.to_numpy().tolist(), tokenizer, max_length, y_test.to_numpy())

tokenized_training_data_synoynm = ProcessData(synoynm_X_train.to_numpy().tolist(), tokenizer, max_length, y_train.to_numpy())
tokenized_validation_data_synoynm = ProcessData(synoynm_X_val.to_numpy().tolist(), tokenizer, max_length, y_val.to_numpy())
tokenized_test_data_synoynm = ProcessData(synoynm_X_test.to_numpy().tolist(), tokenizer, max_length, y_test.to_numpy())

In [8]:
batch_size = 16
params = {'batch_size': batch_size,
          'num_workers': 0}
#Create dataloader
loader_tokenized_training_data = torch.utils.data.DataLoader(tokenized_training_data, **params)
loader_tokenized_training_data_typo = torch.utils.data.DataLoader(tokenized_training_data_typo, **params)
loader_tokenized_training_data_swap = torch.utils.data.DataLoader(tokenized_training_data_swap, **params)
loader_tokenized_training_data_delete = torch.utils.data.DataLoader(tokenized_training_data_delete, **params)
loader_tokenized_training_data_synoynm = torch.utils.data.DataLoader(tokenized_training_data_synoynm, **params)

loaders = [loader_tokenized_training_data, loader_tokenized_training_data_typo, loader_tokenized_training_data_swap, loader_tokenized_training_data_delete, loader_tokenized_training_data_synoynm]

In [9]:

# TODO MANUALLY BATCH DATA

def create_data_loader_iterators(data_loaders):
    result = []
    for data_loader in data_loaders:
        result += [iter(data_loader)]
    return result

def training(model, data_loaders, size, eps):
    model = model.train()
    losses = []
    training_acc = 0
    
    iters = create_data_loader_iterators(data_loaders)
    for _ in tqdm(range(len(data_loaders[0]) - eps)):
       
        batch_ids_list = []
        batch_masks_list = []
        labels = None
        for i in iters:
            data = next(i)
            batch_ids = data[0]['input_ids']
            batch_ids = batch_ids.flatten().reshape((batch_ids.shape[0], batch_ids.shape[2]))
            batch_ids_list.append(batch_ids.to(device))
            batch_masks = data[0]['attention_mask']
            batch_masks = batch_masks.flatten().reshape((batch_masks.shape[0], batch_masks.shape[2]))
            batch_masks_list.append(batch_masks.to(device))
            labels = data[2]
        labels = labels.to(device)
        output = model(batch_ids_list, batch_masks_list)
        prediction = torch.max(output, 1)[1]
        training_loss = criterion(output, torch.flatten(labels))
        training_acc += torch.sum(prediction == torch.flatten(labels))
        losses.append(training_loss.item())
        training_loss.backward()
        #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        # torch.cuda.empty_cache()
    return training_acc / size, np.mean(losses)

def evaluate(model, data_loader, size):
    model = model.eval()
    losses = []
    validation_acc = 0
    with torch.no_grad():
        for vdata in tqdm(data_loader):
            vbatch_ids = vdata[0]['input_ids']
            vbatch_ids = vbatch_ids.flatten().reshape((vbatch_ids.shape[0], vbatch_ids.shape[2]))
            vbatch_masks = vdata[0]['attention_mask']
            vbatch_masks = vbatch_masks.flatten().reshape((vbatch_masks.shape[0], vbatch_masks.shape[2]))
            vdata[2] = vdata[2].to(device)
           

            voutput = model(vbatch_ids.to(device), vbatch_masks.to(device))
            vprediction = torch.max(voutput, 1)[1]
            
            vloss = criterion(voutput, torch.flatten(vdata[2]))
            validation_acc += torch.sum(vprediction == torch.flatten(vdata[2]))
            losses.append(vloss.item())
    return validation_acc / size, np.mean(losses)

DECLARE BATCHES IN TRAINING LOOP

In [10]:
class TransTCN(nn.Module):
    '''
    classes: number of ratings (1-5)
    num_augmentations: number of data augmentations
    input_size: should be the same as num_augmentations.
    num_channels: list of number of in/out channels for each layer. The first layer takes in_channel input_size
                  For ex) [4, 4, 4, 4, 1] = 3 layers: input_size, 4 | 4, 4 | 4, 1 |
    kernel_size: size of the filter in TCN
    dropout: dropout probability for TCN
    hidden_state: default 768, was size of the output of pretrained bert
    is_single_bert: True or False value. If True, uses 1 bert model for all 
                    num_augmentations augementations. Else, uses num_augmentations bert 
                    models for num_augmentations augementations.
    '''
    def __init__(self, classes, num_augmentations, input_size, num_channels, kernel_size=2, dropout=0.3, hidden_state=768, is_single_bert=False):
        super(TransTCN, self).__init__()
        self.num_augmentations = num_augmentations
        self.is_single_bert = is_single_bert
        self.berts = []
        if self.is_single_bert:
            self.berts += [transformers.BertModel.from_pretrained('bert-base-uncased').to(device)]
        else:
            for i in range(num_augmentations):
                self.berts.append(transformers.BertModel.from_pretrained('bert-base-uncased').to(device))
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size, dropout=dropout)
        self.finalLinear = nn.Linear(hidden_state, classes)

    #Return shape (batch_size, num_channels[0], bert_output_dimension)
    def concatBerts(self, bert_outputs):
        concatBert = bert_outputs[0][1].unsqueeze(dim=1)
        for i in range(1, len(bert_outputs)):
            concatBert = torch.cat((concatBert, bert_outputs[i][1].unsqueeze(dim=1)), dim=1)
        return concatBert
    #(bert -> tcn) * n -> bert -> linear -> softmax
    def forward(self, input_ids, attention_masks):
        bert_outputs = []
        for i in range(self.num_augmentations):
            if self.is_single_bert:
                bert_outputs.append(self.berts[0](input_ids[i], attention_masks[i]))
            else:
                bert_outputs.append(self.berts[i](input_ids[i], attention_masks[i]))
        output = self.concatBerts(bert_outputs)
        output = self.tcn(output).squeeze(dim=1)
        output = self.finalLinear(output)
        return output
# [4, 4, 4, 4, 4, 4, 1]
# in_channels, outchannels
# input_size, 4
# 4, 4
# 4, 4
# 4, 1 (batch_size, 1, 768)

In [11]:
num_tcn_layers = 3
model = TransTCN(classes=5, num_augmentations=5, input_size=5,num_channels=[5,5] + [1], kernel_size = 2, is_single_bert=True)
model = model.to(device)
from torch.optim import Adam
criterion = nn.CrossEntropyLoss().to(device)
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
num_epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(loader_tokenized_training_data) * num_epochs)

In [12]:
highest_acc = 0
for epoch in trange(num_epochs):
    print('Epoch: ' , str(epoch))
    print('==================================')
    # If the batch is not batch_size length, the training loop wont run that batch
    eps = len(X_train) % batch_size != 0
    training_accuracy, training_loss = training(model,loaders, len(X_train), eps)
    #validation_accuracy, validation_loss = evaluate(model, loader_tokenized_validation_data, len(X_val))
    
    print('Training accuracy: ', training_accuracy )
    print('Training loss: ', training_loss)
    #print('Validation accuracy: ', validation_accuracy)
    #print('Validation loss: ', validation_loss)

Epoch:  0
creatings loaders


here




RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 8.00 GiB total capacity; 5.17 GiB already allocated; 9.00 MiB free; 6.23 GiB reserved in total by PyTorch)

In [ ]:
print(len(np.where(y_train == 5)[0])/ len(y_train)) 